In [0]:
# Initialization
import numpy as np
seed = 256
np.random.seed(seed)
import tensorflow as tf
tf.random.set_seed(seed)

In [0]:
# Imports
import os
from keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.utils import class_weight
from keras.optimizers import Adam

In [1]:
# Google Drive mount
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# Links functions
%run /content/drive/My\ Drive/segmentation/functions.ipynb
%run /content/drive/My\ Drive/segmentation/models.ipynb
%run /content/drive/My\ Drive/segmentation/metrics.ipynb

Using TensorFlow backend.


In [0]:
# Config
epochs = 40
batch_size = 1
save_dir = '/content/drive/My Drive/segmentation/saved_models/'
model_name = 'UNet3D-spine'
model_dir = os.path.join(save_dir, model_name)
create_data = False

dim_x = 96
dim_y = 96
dim_z = 128

data_path = '/content/drive/My Drive/segmentation/data'
dim = (dim_x, dim_y, dim_z)

In [0]:
# Create data
if create_data:
  create_train_imgs(data_path, dim)
  create_train_masks(data_path, dim)
  create_test_imgs(data_path, dim)
  create_test_masks(data_path, dim)

Train

In [0]:
# Load data
train_imgs, train_masks = load_train_data(data_path)

In [0]:
# Switch vertebrae mask to spine mask
train_masks = vertebrae_to_spine(train_masks)

In [0]:
# Normalization
train_imgs = train_imgs.astype(np.float32) / 255.
train_masks = train_masks.astype(np.float32)

In [0]:
# Checkpoints
model_dir = os.path.join(save_dir, model_name)
if not os.path.exists(model_dir):
    os.mkdir(model_dir)

model_checkpoint = ModelCheckpoint(os.path.join(model_dir, 'weights.h5'), monitor='val_loss', save_best_only=True)
csv_logger = CSVLogger(os.path.join(model_dir, 'logs.txt'), separator=',', append=True)

In [0]:
# Create model
model = unet_3d((dim_x, dim_y, dim_z, 1))
model.compile(optimizer=Adam(lr=0.00001), loss=dice_coef_loss, metrics=[dice_coef])

In [0]:
# Fit model
model.fit(train_imgs, train_masks, batch_size=batch_size, epochs=epochs, verbose=1, shuffle=False, validation_split=0.8, callbacks=[model_checkpoint, csv_logger])

In [0]:
# Plot losses
plot_loss(os.path.join(model_dir, 'logs.txt'))

In [0]:
# Save model 
model.save(os.path.join(model_dir, 'model.h5'))

Predict

In [0]:
# Load test data
test_imgs, test_masks = load_test_data(data_path)

In [0]:
# Prediction
predicted_masks = model.predict(test_imgs, batch_size=1, verbose=1)

In [0]:
save_predictions(model_dir, predicted_masks)

In [0]:
# Mask images
masked_imgs = mask_images(test_imgs, predicted_masks > 0.9)

In [0]:
# Visualize predictions
plot_predictions(0, predicted_masks, test_imgs, masked_imgs)

Accuracy

In [0]:
pixel_wise_accuracy(test_masks, (predicted_masks>0.9))

In [0]:
dice(test_masks, (predicted_masks > 0.9).astype(np.float32))

In [0]:
iou(test_masks, (predicted_masks > 0.9).astype(np.float32))

In [0]:
auc(test_masks, (predicted_masks > 0.9).astype(np.float32))